In [ ]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import torch

import numpy as np
from sklearn.cluster import KMeans
from docx import Document
from docx.shared import Pt, RGBColor
from docx.oxml.shared import OxmlElement, qn
from docx.oxml import ns

from docx.oxml import OxmlElement, ns
from docx.oxml.ns import qn
from docx.opc.constants import RELATIONSHIP_TYPE


### semantic embeddings models

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
 
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large")


### openai models

In [ ]:
import openai
openai.api_key= "YOUR-OPENAI-KEY"

def get_completion(prompt, model): 
    messages = [ {"role": "system", "content": "You are a helpful assistant that respects very precisely the instructions given."},
                {"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]



### variables

In [7]:
df = pd.read_csv("Articles.csv")

In [8]:
df["Vector"] = 0

In [9]:
df

,Titre,Resume,Date,Url,Website,politique,géopolitique,sociologique,démographie,économie micro,...,International,Monde,Industrie,Marque et enseigne,Services,Energie,Construction,Utilities,Mobilité,Vector
0,Un journal non aligné,"Le Monde diplomatique, un journal non aligné, ...",2023-11-01,https://www.monde-diplomatique.fr/2023/11/BREV...,Monde Diplomatique,Non,Non,Non,Non,Non,...,Oui,Oui,Non,Non,Non,Oui,Oui,Non,Non,0
1,Un renoncement français,"L'article intitulé ""Il n'y aura pas de rue Emm...",2023-11-01,https://www.monde-diplomatique.fr/2023/11/BREV...,Monde Diplomatique,Non,Non,Non,Non,Non,...,Oui,Oui,Non,Non,Non,Oui,Oui,Non,Non,0
2,Quand la guerre percute la politique française,Le nouveau conflit au Proche-Orient a eu un im...,2023-11-01,https://www.monde-diplomatique.fr/2023/11/HALI...,Monde Diplomatique,Non,Non,Non,Non,Non,...,Oui,Oui,Non,Non,Non,Oui,Oui,Non,Non,0
3,"Loin du front, la société ukrainienne coupée e...","Dans un reportage intitulé ""Loin du front, la ...",2023-11-01,https://www.monde-diplomatique.fr/2023/11/RICH...,Monde Diplomatique,Non,Non,Non,Non,Non,...,Oui,Oui,Non,Non,Non,Oui,Oui,Oui,Non,0
4,"Israël-Palestine, l’engrenage guerrier","L'article ""Israël-Palestine, l'engrenage guerr...",2023-11-01,https://www.monde-diplomatique.fr/2023/11/BELK...,Monde Diplomatique,Non,Non,Non,Non,Non,...,Oui,Oui,Non,Non,Non,Oui,Oui,Non,Non,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,Blue bonds : de nouvelles lignes directrices p...,L'association internationale des marchés finan...,2023-11-07,https://lessentiel.novethic.fr/blog/business-c...,L'essentiel Novethic,Non,Non,Non,Non,Non,...,Oui,Oui,Non,Non,Non,Non,Non,Non,Non,0
586,"Titane, lithium : l'Union européenne ouvre un ...",La future législation européenne sur les matiè...,2023-11-07,https://reporterre.net/Titane-lithium-l-Union-...,Reporterre,Oui,Non,Non,Non,Non,...,Oui,Oui,Oui,Non,Non,Non,Non,Non,Non,0
587,La plus haute ZAD d'Europe remet en cause l'am...,Des militants écologistes ont occupé le glacie...,2023-11-07,https://www.latribune.fr/opinions/tribunes/la-...,La Tribune,Non,Non,Non,Non,Non,...,Non,Non,Non,Non,Non,Non,Non,Non,Non,0
588,"La crise au Proche-Orient, une aubaine pour la...",La crise au Proche-Orient profite à la Russie ...,2023-11-07,https://www.latribune.fr/opinions/tribunes/la-...,La Tribune,Oui,Oui,Non,Non,Non,...,Oui,Oui,Non,Non,Non,Non,Non,Non,Non,0


### Embedding

In [ ]:
def vectorize(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
   
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [ ]:
def vectorize_resume(resume,model,tokenizer):
    
    vector = vectorize(resume, tokenizer, model)
        

    if not isinstance(vector, str):
        vector = ' '.join(map(str, vector))  
    
    
    return vector

In [ ]:
for index, row in df.iterrows():
    
    if str(row['Vector']).strip()[0].isdigit():
        
        df.at[index, 'Vector'] = vectorize_resume(row['Resume'], model, tokenizer)
        
        # Sauvegarder le DataFrame après chaque mise à jour de ligne (si nécessaire)
        df.to_csv("Articles_Vectorized.csv", index=False, encoding='utf-8-sig')
        
### On pourrait utiliser la méthode apply mais comme le run est long sur CPU (SIM-CSE est un modèle à plus 1 Go),
### On s'assure qu'on sauvegarde à chaque calcul


### A ce stade, on a projeté les résumés dans un grand espace vectoriel capable de capturer sémantiquement les articles

### Le but est de regrouper les articles par thème, en définissant un seuil dans les clusters qui permet de dire si oui ou non deux articles sont du même thème, ce seuil est subjectif, plus il est petit plus les articles vont dans le même sens

In [ ]:
df_vec = pd.read_csv("Articles_Vectorized.csv", index=False, encoding='utf-8-sig')


### Algo (threshold à 11.5)

In [ ]:
def find_optimal_clusters(df, data, threshold):
    remaining_data = data.copy() ### Les vecteurs provenant de la projection des résumés par l'encodeur sémantique
    remaining_df = df.copy() ### dataframe à néttoyer
    optimal_clusters = []
    cluster_number = 0

    while remaining_data.shape[0] > 0:
        k = 1 ###Nombre de clusters initialisé à 1 à chaque fois qu'on en trouve un nouveau, ce qui augmente la complexité
        found = False

        while not found:
            kmeans = KMeans(n_clusters=k, random_state=0).fit(remaining_data) ###Algo KMeans
            cluster_centers = kmeans.cluster_centers_
            labels = kmeans.labels_

            
            for i in range(k): ###parcours des clusters
                
                cluster_mask = labels == i
                cluster_data = remaining_data[cluster_mask]
                centroid = np.mean(cluster_data, axis=0)

                
                distances = np.linalg.norm(cluster_data - centroid, axis=1)
                
                


                if np.all(distances < threshold): ### Si un cluster est assez petit, on le garde
                    max_distance = np.max(distances)
                    print(max_distance)
                    print("trouvé")
                    found = True
                    cluster_number += 1  
                    
                    intra_cluster_distances = np.linalg.norm(cluster_data - cluster_data[:, np.newaxis], axis=2)
                    representative_index = np.argmin(np.mean(intra_cluster_distances, axis=1))

                    cluster_df = remaining_df.iloc[cluster_mask]
                    representative_title = cluster_df.iloc[representative_index]['Title']
                    cluster_info = { ##de la mise en forme pour le dataframe final
                        "Numéro du thème": cluster_number, 
                        "Url": cluster_df['Url'].tolist(),
                        "Date": cluster_df['Date'].tolist(),
                        "Resume": cluster_df['Resume'].tolist(),
                        "Titre": cluster_df['Title'].tolist(),
                        "Représentant": representative_title,
                        
                    }
                    optimal_clusters.append(cluster_info)
                    print(cluster_info)

                    
                    remaining_data = remaining_data[~cluster_mask] ###On enlève les vecteurs qui appartiennent au cluster
                    remaining_df = remaining_df.iloc[~cluster_mask]
                    print(remaining_df.shape)
                    break

            if not found: ### Si aucun cluster assez petit trouvé On augmente le nombre de clusters possibles dans le K-Means
                k += 1

    return optimal_clusters


In [11]:
def segmentation_articles(df,threshold):
    
    df['Vector'] = df['Vector'].apply(lambda x: np.fromstring(x, dtype=float, sep=' '))
    
    data = np.array(df['Vector'].tolist())
    print(data)
    clusters_info = find_optimal_clusters(df, data, threshold=threshold)
    
    clusters_df = pd.DataFrame(clusters_info)
    expanded_clusters_df = clusters_df.apply(pd.Series.explode 
                                             if not clusters_df.empty else clusters_df)

    expanded_clusters_df["Représentant"] = expanded_clusters_df["Représentant"].astype(str)
    expanded_clusters_df["Titre"] = expanded_clusters_df["Titre"].astype(str)
    
    
    result = []


    for _, row in expanded_clusters_df.iterrows():
   
        if row["Titre"] == row["Représentant"]:
            result.append("oui")
        else:
            result.append("non")

            
    expanded_clusters_df["Est-il le représentant du thème ?"] = result
    

    expanded_clusters_df['Nombre d\'articles thème'] = expanded_clusters_df.groupby('Numéro du thème')['Resume'].transform('count')
    
    
    

    
    
    merged_df = expanded_clusters_df.merge(df, on="Resume", how="left", suffixes=('', '_dup'))


    for col in df.columns:
        if col != "Resume" and (col + '_dup') in merged_df.columns:
            merged_df.drop(col + '_dup', axis=1, inplace=True)
    
    merged_df = merged_df.drop(columns=['Vector','Title'])



    
    
    return merged_df


    
    
    
    


In [ ]:
df_final = segmentation_articles(df_vec,11.5)

### Ecriture de la Newsletter

### Filtre sur les plus gros clusters (thème d'actualité)

In [ ]:
theme_dict = defaultdict(dict)

for _, row in df_final.iterrows():
    if row["Nombre d'articles thème"] >= 6:
           
        theme = row['Numéro du thème']
        url = row['Url']
        resume = row['Resume']
    

    theme_dict[theme][url] = resume


theme_dict = dict(theme_dict)


reordered_theme_dict = {}

for new_id, old_id in enumerate(sorted(theme_dict.keys()), start=1):
    reordered_theme_dict[new_id] = theme_dict[old_id]


theme_dict = reordered_theme_dict

In [ ]:
representant_dict = defaultdict(dict)

for _, row in df_finalv2.iterrows():
    if row["Nombre d'articles thème"] >= 6:
           
        if row['Est-il le représentant du thème ?'] == "oui":
            theme = row['Numéro du thème']
            titre = row['Titre']
            representant_dict[theme] = titre


representant_dict = dict(representant_dict)


reordered_representant_dict= {}

for new_id, old_id in enumerate(sorted(representant_dict.keys()), start=1):
    reordered_representant_dict[new_id] = representant_dict[old_id]

    
representant_dict = reordered_representant_dict 

### Synthèse des clusters par gpt-4 et mise en forme Word

In [ ]:
def docx_add_hyperlink(paragraph, text, url, doc, is_external=True):
    
    run = paragraph.add_run(text)
    run.font.size = Pt(12)  
    run.font.color.rgb = RGBColor(0, 0, 255)
    run.font.underline = True

    hyperlink = OxmlElement('w:hyperlink')
    id = doc.part.relate_to(url, RELATIONSHIP_TYPE.HYPERLINK, is_external=is_external)
    hyperlink.set(qn('r:id'), id)

  
    run._r.getparent().replace(run._r, hyperlink)
    hyperlink.append(run._r)
    
    return run


In [ ]:
def find_preceding_word(text, start_index, end_index):
    
    word_pattern = re.compile(r'(\b\w+\b"?\s?[?;!.>»]?|!"|\?")(?=\s*\(?https)')

    
    match = word_pattern.search(text[start_index:end_index])
    
    if match:
    
        return match.group(1), match.start(1)
    return None, None


In [ ]:
def create_doc_with_hyperlinks(theme_dict, representing_dict, fichier):
    url_pattern = r"(?<=\()?(https?://[^\s)]+)(?=\))?"


    doc = Document()
    
    for k in range(1, len(theme_dict) + 1):
        doc.add_heading(representing_dict[k], level=1)
        done = False 
        while not(done):
            try :
                prompt_final = f""" En adoptant un ton humoristique et légèrement décalé qui soit adapté, narre en 200 mots maximum une synthèse de revue de presse à la troisième personne.
                Focus sur deux ou trois articles essentiels, en citant leurs URLs en entièreté, tirés de << {theme_dict[k]} >> """
                bullet_texte = get_completion(prompt_final, model = "gpt-4")
                done = True
            except Exception as e:
                print(e)
        
            
                                      
                                      
       
        
       
        urls = re.findall(url_pattern, bullet_texte)
        
        



        start_index = 0
        p = doc.add_paragraph()

        
        print(bullet_texte)
        print(urls)



        for url in urls:
            print(start_index)
    
            end_index = bullet_texte.rfind(url, start_index)
            print(end_index)
        
        
            if bullet_texte[end_index-1] == "(":
                end_index += 1 
            end_index += len(url)
        
        
 
    
            preceding_word, word_start = find_preceding_word(bullet_texte, start_index, end_index)
            
            
    
            end_word = bullet_texte[:end_index-len(url)].rfind(preceding_word, start_index)
            print(end_word)
        
    
            p.add_run(bullet_texte[start_index:end_word])
    
            print(bullet_texte[start_index:end_word])
            
            docx_add_hyperlink(p, preceding_word, url, doc)
    
    
            start_index = end_index
            if end_index < len(bullet_texte) and bullet_texte[end_index] == ')':
                end_index += 1

        if end_index < len(bullet_texte):
            p.add_run(bullet_texte[end_index:])
            
        doc.save(fichier)

    

    
 


In [ ]:
fichier = f"Revue de presse.docx"
create_doc_with_hyperlinks(theme_dict, representant_dict, fichier)